# Imports

In [4]:
import os
import numpy as np
os.environ["SAGE_NUM_THREADS"] = '64' #set to number of cores

In [5]:
run create_generic_polynomial.ipynb

to create a generic polynomial with 𝐷 indeterminates with generic coefs

In [6]:
run get_all_polynomial_conserved_functions.ipynb 

to get all polynomial conservation laws for given vector fields

In [7]:
run get_specific_vector_fields_and_known_conserved_functions.ipynb

to get 
- the vector fields asociated to linear and ReLU neural networs and 
- the number of independent conserved functions already known in these cases

In [8]:
run computation_dimension_trace_Lie_algebra.ipynb # to get the dim of Lie(V)(x)

In [9]:
sage.parallel.ncpus.ncpus()

64

In [10]:
# increase the maximum memory usage
# warning: this can consume a lot of RAM
maxima._eval_line(":lisp (ext:set-limit 'ext:heap-size 100000000000)",
wait_for_prompt=False)

# Test on specific vector fields $v_i$ (e.g. $v_i:= \nabla \phi_i$).
**In this notebook we test our results on different vector fields ($(v_i)_{i = 1, \cdots, d}$), from simple examples to more sophisticated ones. For ReLU and linear neural networks, the vector fields ($(v_i)_{i = 1, \cdots, d}$) associated to the model $\phi$ (then $v_i = \nabla \phi_i$) are build in the notebook run get_specific_vector_fields_and_known_conserved_functions.ipynb, as well as the number $N$ of independent conservation laws already known by the literature. We systemically compute $\mathrm{dim} \mathrm{Lie} V_\phi (x)$ (via the notebook computation_dimension_trace_Lie_algebra.ipynb) and we check if $D- \mathrm{dim} \mathrm{Lie} V_\phi (x) = N$: if so, we know thanks to Corollary 3.4 that there are no more conserved functions. Finally we check that all polynomial conservation laws we get (via the notebook get_all_polynomial_conserved_functions.ipynb) correspond exactly to the known conservation laws, for ReLU and linear cases.**

### Table of contents: 
- **Part A: Test when $(v_i)_i$ are random polynomial vector fields**  
- **Part B: Test on the simple case: $\phi(x) = x_1 \times \cdots \times x_D$**  
- **Part C: Test on the $q$-layer linear networks (see Section 4 of our paper)**  
- **Part D: Test on the $q$-layer ReLU networks (with or without bias)** 



## A. Test on $d$ random polynomial of degree $m$ vector fields.

In [11]:
def build_vector_fields(D, d, m):
    list_var = [var('x'+str(i+1)) for i in range(D)]
    R = PolynomialRing(ZZ, list_var) 
    return np.array([[R.random_element(degree = m) for _ in range(D)] for _ in range(d)]).T

In [12]:
D = 5
random_vector_fields = build_vector_fields(D, 1, 3)
solve(0, random_vector_fields, True) # the kernel dimension is 1 but the constant polynomial is not counted because
# its gradient is zero so is not a linearly independent vector

# the first argument of solve is the degree of conserved poynomial we are looking for

([1.00000000000000], 0)

In [13]:
solve(4, random_vector_fields, True)  # in general, there are no polynomial conserved functions for random polynomial vector fields

0

In [14]:
D - dim_trace_lie_algebra(random_vector_fields, fast=False)  # number of independent conserved laws in that case

4

## B. Test on $\phi(x) = x_1 \times \cdots \times x_D$ (*i.e.*  $d = 1$ and $v(x)= \nabla \phi (x)$).

In [15]:
def particular_vf(D):
    liste_var = [var('x'+str(i+1)) for i in range(D)]
    R = PolynomialRing(ZZ, liste_var) 
    list_ = [1]*D
    Q = R.monomial(*list_)
    return np.array(Q.gradient()).reshape(D, 1)

In [16]:
particular_vf(3)

array([[x2*x3],
       [x1*x3],
       [x1*x2]], dtype=object)

We obtain that there are $D-1$ independent homogeneous polynomial of degree 2 conserved functions (and there are no more conserved functions as $(D-1)+ 1 = D$).

In [17]:
V2 = particular_vf(3)
solve(2, V2, True) 

([-x1^2 + x3^2, -x1^2 + x2^2], 2)

Let's check that the dimension of Lie(V)(x) is equal to 1 !

In [18]:
dim_trace_lie_algebra(V2, fast=False)

1

## C. Test on the case of $q$-layer linear neural networks $\phi(U_1, \cdots, U_q) = U_1 \times \cdots U_q$ ($v_i:= \nabla \phi_i(U_1, \cdots, U_q)$).

In [17]:
from tqdm import tqdm

In [23]:
np.random.seed(0)
for i in tqdm(range(50)):
    depth = np.random.randint(2, 5)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 6)
    list_dim = list(list_dim)
    n_known_functions = known_conserved_functions_linear(list_dim)
    D, vector_fields = vector_fields_for_q_layer_LNN(list_dim)
    dim_found = D - dim_trace_lie_algebra(vector_fields, n_known_functions=n_known_functions, verbose=0)
    assert dim_found == n_known_functions, "dim_found = {}, n_known_functions = {}, for list_dim {}".format(dim_found, n_known_functions, list_dim)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [34:23<00:00, 41.26s/it]


The number of independent conserved functions that are already known matches with $D - \text{dim} Lie V_{\phi} (\theta)$: there is no other conservation law!

In [19]:
np.random.seed(0)
for i in tqdm(range(10)):
    depth = np.random.randint(2, 4)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 5)
    list_dim = list(list_dim)
    D, vector_fields = vector_fields_for_q_layer_LNN(list_dim)
    n_known_functions = known_conserved_functions_linear(list_dim)
    assert solve(2, vector_fields) == n_known_functions

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [01:31<00:00,  9.16s/it]


As expected, we can also find all known conserved functions by looking for polynomial conserved functions of degree 2, using our algorithm presented in Section 2.4 of our paper.

## D. Test on the case of $q$-layer ReLU neural networks.

In [20]:
np.random.seed(0)
for i in tqdm(range(50)):
    depth = np.random.randint(2, 5)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 10)
    list_dim = list(list_dim)
    if np.random.random() < 0.5:
        bias = True
    else:
        bias = False
    n_known_functions = known_conserved_functions_ReLU(list_dim)
    D, vector_fields = vector_fields_ReLU_q_layers(list_dim, bias=bias)
    dim_found = D - dim_trace_lie_algebra(vector_fields, n_known_functions=n_known_functions, verbose=0)
    assert dim_found == n_known_functions, "dim_found = {}, n_known_functions = {}, for list_dim {}".format(dim_found, n_known_functions, list_dim)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 50/50 [1:53:11<00:00, 135.84s/it]


The number of independent conserved functions that are already known matches with $D - \text{dim} Lie V_{\phi} (\theta)$: there is no other conservation law!

In [21]:
np.random.seed(0)
for i in tqdm(range(10)):
    depth = np.random.randint(2, 4)
    list_dim = np.empty(depth + 1, dtype=int)
    for j in range(depth + 1):
        list_dim[j] = np.random.randint(2, 5)
    list_dim = list(list_dim)
    n_known_functions = known_conserved_functions_ReLU(list_dim)
    D, vector_fields = vector_fields_ReLU_q_layers(list_dim)
    assert solve(2, vector_fields) == n_known_functions


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:44<00:00,  4.49s/it]


As expected, we can also find all known conserved functions by looking for polynomial conserved functions of degree 2, using our algorithm presented in Section 2.4 of our paper.